This notebook can be used to calculate NA Corrected intensities as well as fractional enrichment for LCMS data with resolution varying with molecular mass. The example shows a dataset with C13 and N15 label:

 - auto_detect_indistinguish_dual_label.csv - demo raw MS intensity file containing intensities for C10H17N3O6S simulated using combinatorics by considering C13 indistinguishable with O17 and N15 indistinguishable with S34

In [1]:
import pandas as pd
import numpy as np
import re

from corna.inputs import maven_parser as parser
import corna.constants as const
from corna.helpers import replace_negatives_in_column, merge_multiple_dfs
from corna.algorithms.nacorr_lcms import na_correction
from corna.postprocess import fractional_enrichment


Reading raw file and merging with sample metadata if present, in this example running without sample metadata

In [2]:
raw_df = pd.read_csv('auto_detect_indistinguish_dual_label.csv')
sample_metadata = pd.DataFrame()

merged_df, iso_tracer_data, element_list = parser.read_maven_file(raw_df, sample_metadata)
merged_df.head()

,Name,Label,Formula,Sample,Intensity,Unlabeled Fragment
0,Glutathione,C12 PARENT,C10H17N3O6S,Sample 1,0.000,Glutathione
1,Glutathione,C13-label-1,C10H17N3O6S,Sample 1,0.000,Glutathione
2,Glutathione,C13-label-2,C10H17N3O6S,Sample 1,0.000,Glutathione
3,Glutathione,C13-label-3,C10H17N3O6S,Sample 1,0.178,Glutathione
4,Glutathione,C13-label-4,C10H17N3O6S,Sample 1,0.013,Glutathione


Dictionary containing natural abundance values for the common isotopes found in nature. It can be defined by the user or one can use the default values from the package. The format of the dictionary is as shown below: 

{E:[M0, M1, ..Mn]} where E is the element symbol and the natural abundance fraction is in the increasing order of masses. For example:

In [3]:
#user defined
na_dict={'C':[0.9889,0.0111],
           'H':[0.99985, 0.00015],
           'N':[0.9964,0.0036],
           'O':[0.9976,0.0004,0.002],
           'S':[0.950,0.0076,0.0424]}

Performing na_correction and using the dictionary above for NA values. For Orbitrap, for molecular mass of 307 (which is our input compound) and 293808 resolution, ppm is ~7, according to the formula from Su, Xiaoyang et al.,2017

\begin{equation*}
\frac{Δm}{m} = 1.66 × \frac{m^\frac{1}{2}}{(MinimalNominalResolution×√200)} × 10^6
\end{equation*}

which is our ppm_user_input 

In [4]:
na_corr_df, ele_corr_dict = na_correction(merged_df, iso_tracers=['C13', 'N15'], ppm_input_user=7, 
                                          eleme_corr={}, na_dict=na_dict, autodetect=True)
na_corr_df = replace_negatives_in_column(na_corr_df, const.NA_CORRECTED_WITH_ZERO, const.NA_CORRECTED_COL)
na_corr_df


,Name,Formula,Indistinguishable_isotope,Sample,NA Corrected,Intensity,Label,NA Corrected with zero
0,Glutathione,C10H17N3O6S,"{'C': ['O17'], 'N': ['S34']}",Sample 1,9.867204e-17,0.0000,C12 PARENT,9.867204e-17
1,Glutathione,C10H17N3O6S,"{'C': ['O17'], 'N': ['S34']}",Sample 1,3.311852e-06,0.0000,N15-label-1,3.311852e-06
2,Glutathione,C10H17N3O6S,"{'C': ['O17'], 'N': ['S34']}",Sample 1,-2.384534e-08,0.0000,N15-label-2,0.000000e+00
3,Glutathione,C10H17N3O6S,"{'C': ['O17'], 'N': ['S34']}",Sample 1,8.128195e-01,0.6808,N15-label-3,8.128195e-01
4,Glutathione,C10H17N3O6S,"{'C': ['O17'], 'N': ['S34']}",Sample 1,6.476387e-17,0.0000,C13-label-1,6.476387e-17
5,Glutathione,C10H17N3O6S,"{'C': ['O17'], 'N': ['S34']}",Sample 1,-1.305946e-07,0.0000,C13N15-label-1-1,0.000000e+00
6,Glutathione,C10H17N3O6S,"{'C': ['O17'], 'N': ['S34']}",Sample 1,9.402814e-10,0.0000,C13N15-label-1-2,9.402814e-10
7,Glutathione,C10H17N3O6S,"{'C': ['O17'], 'N': ['S34']}",Sample 1,5.284966e-04,0.0785,C13N15-label-1-3,5.284966e-04
8,Glutathione,C10H17N3O6S,"{'C': ['O17'], 'N': ['S34']}",Sample 1,2.193459e-17,0.0000,C13-label-2,2.193459e-17
9,Glutathione,C10H17N3O6S,"{'C': ['O17'], 'N': ['S34']}",Sample 1,-6.844887e-07,0.0000,C13N15-label-2-1,0.000000e+00


Calculating fractional enrichments, merging all data into a single file and saving as 'auto_detect_dual_label_isotope_ppm7.csv'

In [5]:
frac_enr_df = fractional_enrichment(na_corr_df)
frac_enr_df

,Sample,Name,Label,Formula,Pool_total,Fractional enrichment
0,Sample 1,Glutathione,C12 PARENT,C10H17N3O6S,1.023658,9.639160e-17
1,Sample 1,Glutathione,N15-label-1,C10H17N3O6S,1.023658,3.235311e-06
2,Sample 1,Glutathione,N15-label-2,C10H17N3O6S,1.023658,0.000000e+00
3,Sample 1,Glutathione,N15-label-3,C10H17N3O6S,1.023658,7.940342e-01
4,Sample 1,Glutathione,C13-label-1,C10H17N3O6S,1.023658,6.326709e-17
5,Sample 1,Glutathione,C13N15-label-1-1,C10H17N3O6S,1.023658,0.000000e+00
6,Sample 1,Glutathione,C13N15-label-1-2,C10H17N3O6S,1.023658,9.185503e-10
7,Sample 1,Glutathione,C13N15-label-1-3,C10H17N3O6S,1.023658,5.162824e-04
8,Sample 1,Glutathione,C13-label-2,C10H17N3O6S,1.023658,2.142765e-17
9,Sample 1,Glutathione,C13N15-label-2-1,C10H17N3O6S,1.023658,0.000000e+00


In [6]:
output_df = merge_multiple_dfs([merged_df, na_corr_df, frac_enr_df])
output_df

,Name,Label,Formula,Sample,Intensity_x,Unlabeled Fragment,Indistinguishable_isotope,NA Corrected,Intensity_y,NA Corrected with zero,Pool_total,Fractional enrichment
0,Glutathione,C12 PARENT,C10H17N3O6S,Sample 1,0.0000,Glutathione,"{'C': ['O17'], 'N': ['S34']}",9.867204e-17,0.0000,9.867204e-17,1.023658,9.639160e-17
1,Glutathione,C13-label-1,C10H17N3O6S,Sample 1,0.0000,Glutathione,"{'C': ['O17'], 'N': ['S34']}",6.476387e-17,0.0000,6.476387e-17,1.023658,6.326709e-17
2,Glutathione,C13-label-2,C10H17N3O6S,Sample 1,0.0000,Glutathione,"{'C': ['O17'], 'N': ['S34']}",2.193459e-17,0.0000,2.193459e-17,1.023658,2.142765e-17
3,Glutathione,C13-label-3,C10H17N3O6S,Sample 1,0.1780,Glutathione,"{'C': ['O17'], 'N': ['S34']}",2.077735e-01,0.1780,2.077735e-01,1.023658,2.029716e-01
4,Glutathione,C13-label-4,C10H17N3O6S,Sample 1,0.0130,Glutathione,"{'C': ['O17'], 'N': ['S34']}",-1.632273e-03,0.0130,0.000000e+00,1.023658,0.000000e+00
5,Glutathione,C13-label-5,C10H17N3O6S,Sample 1,0.0002,Glutathione,"{'C': ['O17'], 'N': ['S34']}",-2.356019e-04,0.0002,0.000000e+00,1.023658,0.000000e+00
6,Glutathione,C13-label-6,C10H17N3O6S,Sample 1,0.0001,Glutathione,"{'C': ['O17'], 'N': ['S34']}",1.185349e-04,0.0001,1.185349e-04,1.023658,1.157954e-04
7,Glutathione,C13N15-label-1-3,C10H17N3O6S,Sample 1,0.0785,Glutathione,"{'C': ['O17'], 'N': ['S34']}",5.284966e-04,0.0785,5.284966e-04,1.023658,5.162824e-04
8,Glutathione,C13N15-label-1-1,C10H17N3O6S,Sample 1,0.0000,Glutathione,"{'C': ['O17'], 'N': ['S34']}",-1.305946e-07,0.0000,0.000000e+00,1.023658,0.000000e+00
9,Glutathione,C13N15-label-1-2,C10H17N3O6S,Sample 1,0.0000,Glutathione,"{'C': ['O17'], 'N': ['S34']}",9.402814e-10,0.0000,9.402814e-10,1.023658,9.185503e-10


In [7]:
output_df.to_csv('auto_detect_dual_label_isotope_ppm7.csv')